In [465]:
%matplotlib notebook
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import plotly.express as px
from pylab import rcParams
from datetime import timedelta
import time
import warnings
warnings.filterwarnings('ignore')

# Read and Explore Data

## input

In [466]:
inputfile="C:/Users/changjin.feng/Desktop/dc/DC_UK.xlsx"
inputfile_NL="C:/Users/changjin.feng/Desktop/dc/DC_NL.xlsx"
dep_gate='LGW'
dest_gate='PMI'
rcParams['figure.figsize'] =40, 10
n_sim=1
ratio=2

In [467]:
data_cax=pd.read_excel(inputfile,sheet_name='S19 Flt CAX')
data_cax=data_cax.loc[(data_cax['GATEWAY_CODE_DEPARTURE']==dep_gate)&(data_cax['GATEWAY_CODE_ARRIVAL']==dest_gate),:]
data_cax.head(10)

,SEASON_YEAR,SEASON_TYPE,TRANSPORT_CODE,GATEWAY_CODE_DEPARTURE,GATEWAY_CODE_ARRIVAL,CYCLE_START_DATE,TRANSPORT_DEPARTURE_DATETIME,TRANSPORT_ARRIVAL_DATETIME,DIVISION_CODE,COMPANY_CODE,...,SOURCE_SYSTEM_ID,SOURCE_UPDATED_DATE,LOADED_DATE,CYRUS_TRS_ALLOCATION,FLIGHT_CARRIER,FLIGHT_FLAG,GUARANTEED_ALLOCATION,ALLOCATION_INDICATOR,RELEASE_DATE,VOL_DEAL_IND
72,2019,S,PMILGW6A,LGW,PMI,2019-10-12,2019-10-12 08:30:00,2019-10-12 11:45:00,G,NaN,...,A,10-OCT-19 15.03.39.000000000,11-OCT-19 00.54.58.825898000,325,NaN,NaN,NaN,NaN,NaN,NaN
73,2019,S,PMILGW6C,LGW,PMI,2019-06-08,2019-06-08 06:30:00,2019-06-08 09:25:00,G,NaN,...,A,06-JUN-19 18.42.00.000000000,07-JUN-19 00.49.56.377643000,325,NaN,NaN,NaN,NaN,NaN,NaN
74,2019,S,PMILGW6E,LGW,PMI,2019-07-13,2019-07-13 22:20:00,2019-07-14 01:35:00,G,NaN,...,A,13-JUL-19 12.03.24.000000000,14-JUL-19 00.43.54.174155000,189,NaN,NaN,NaN,NaN,NaN,NaN
105,2019,S,PMILGW3B,LGW,PMI,2019-07-24,2019-07-24 20:30:00,2019-07-24 23:45:00,G,NaN,...,A,24-JUL-19 19.03.15.000000000,25-JUL-19 00.41.47.138417000,221,NaN,NaN,NaN,NaN,NaN,NaN
106,2019,S,PMILGW3C,LGW,PMI,2019-05-08,2019-05-08 20:30:00,2019-05-08 23:45:00,G,NaN,...,A,07-MAY-19 13.03.36.000000000,08-MAY-19 01.45.42.364114000,221,NaN,NaN,NaN,NaN,NaN,NaN
107,2019,S,PMILGW3C,LGW,PMI,2019-08-14,2019-08-14 05:35:00,2019-08-14 09:00:00,G,NaN,...,A,08-AUG-19 14.03.46.000000000,09-AUG-19 00.46.49.324601000,221,NaN,NaN,NaN,NaN,NaN,NaN
108,2019,S,PMILGW3C,LGW,PMI,2019-09-04,2019-09-04 05:35:00,2019-09-04 09:00:00,G,NaN,...,A,03-SEP-19 00.23.41.000000000,04-SEP-19 00.47.40.527256000,221,NaN,NaN,NaN,NaN,NaN,NaN
109,2019,S,PMILGW5A,LGW,PMI,2019-08-30,2019-08-30 21:45:00,2019-08-31 01:00:00,G,NaN,...,A,30-AUG-19 18.03.26.000000000,31-AUG-19 00.49.59.732038000,221,NaN,NaN,NaN,NaN,NaN,NaN
110,2019,S,PMILGW6A,LGW,PMI,2019-06-01,2019-06-01 08:30:00,2019-06-01 11:45:00,G,NaN,...,A,31-MAY-19 18.03.02.000000000,01-JUN-19 00.46.22.638422000,305,NaN,NaN,NaN,NaN,NaN,NaN
111,2019,S,PMILGW6A,LGW,PMI,2019-08-31,2019-08-31 08:30:00,2019-08-31 11:45:00,G,NaN,...,A,28-AUG-19 00.23.50.000000000,29-AUG-19 00.45.17.748869000,305,NaN,NaN,NaN,NaN,NaN,NaN


In [468]:
data_booking_UK=pd.read_excel(inputfile,sheet_name='Historic Bookings')
data_booking_UK=data_booking_UK.loc[(data_booking_UK['GATEWAY_CODE_DEPARTURE']==dep_gate)&(data_booking_UK['GATEWAY_CODE_ARRIVAL']==dest_gate),:]
data_booking_UK=data_booking_UK.loc[data_booking_UK['CYCLE_START_DATE']>=data_booking_UK['BOOKING_DATE'],:]
data_booking_UK=data_booking_UK[['BOOKED_DURATION','CYCLE_START_DATE','BOOKING_DATE','PAX']]
data_booking_UK.head(10)

,BOOKED_DURATION,CYCLE_START_DATE,BOOKING_DATE,PAX
4062,7,2019-05-01,2018-07-15,4
4063,7,2019-05-01,2018-07-27,4
4064,7,2019-05-01,2018-08-27,2
4065,7,2019-05-01,2018-09-21,2
4066,7,2019-05-01,2018-10-15,2
4067,7,2019-05-01,2018-10-16,2
4068,7,2019-05-01,2018-10-17,2
4069,7,2019-05-01,2018-10-28,4
4070,7,2019-05-01,2018-11-02,2
4071,7,2019-05-01,2018-12-30,3


In [469]:
data_dcax_UK=pd.read_excel(inputfile,sheet_name='S19 Flt Cax by Dur ')
data_dcax_UK=data_dcax_UK.loc[(data_dcax_UK['GATEWAY_CODE_DEPARTURE']==dep_gate)&(data_dcax_UK['GATEWAY_CODE_ARRIVAL']==dest_gate),:]
data_dcax_UK=data_dcax_UK[['CYCLE_START_DATE','DURATION','TOTAL_ALLOCATION','BOOKED_ALLOCATION']]
data_dcax_UK.head(10)

,CYCLE_START_DATE,DURATION,TOTAL_ALLOCATION,BOOKED_ALLOCATION
40,2019-07-20,4,3,3
41,2019-07-20,0,4,4
42,2019-08-03,11,26,26
43,2019-08-10,10,30,30
44,2019-08-24,4,4,4
45,2019-08-24,10,30,30
46,2019-07-20,0,47,47
47,2019-07-27,7,184,184
48,2019-07-27,14,32,32
766,2019-10-19,0,21,21


In [470]:
pvt = data_dcax_UK.pivot_table(index =['CYCLE_START_DATE'], 
                     columns = 'DURATION', 
                     values = 'TOTAL_ALLOCATION', 
                     aggfunc = sum).reset_index()
pvt=pvt.fillna(0)
pvt=pvt.set_index('CYCLE_START_DATE')
pvt

DURATION,0,3,4,7,8,10,11,14,21,28
CYCLE_START_DATE,,,,,,,,,,
2019-05-01,0.0,0.0,11.0,160.0,0.0,0.0,24.0,26.0,0.0,0.0
2019-05-04,2.0,0.0,0.0,165.0,0.0,0.0,0.0,22.0,0.0,0.0
2019-05-05,10.0,6.0,0.0,283.0,0.0,24.0,0.0,22.0,0.0,0.0
2019-05-07,0.0,0.0,0.0,116.0,0.0,0.0,33.0,37.0,0.0,0.0
2019-05-08,2.0,0.0,0.0,155.0,0.0,10.0,32.0,22.0,0.0,0.0
2019-05-11,6.0,6.0,6.0,347.0,0.0,33.0,28.0,77.0,2.0,0.0
2019-05-12,6.0,6.0,0.0,272.0,0.0,27.0,0.0,34.0,0.0,0.0
2019-05-14,10.0,0.0,6.0,116.0,0.0,0.0,32.0,22.0,0.0,0.0
2019-05-15,4.0,1.0,13.0,125.0,0.0,36.0,17.0,25.0,0.0,0.0


# Functions

## Preprocessing Distributions

* df0_perc: book month distribution
* df1_perc: departure week distribution per book month
* df2_perc: duration distribution per book month & departure week
* df3_perc: duration distribution per departure week (helper for df2_perc)

In [471]:
def preprocessing (data_booking): 
    
    # filter out negative bookings
    data_booking=data_booking.loc[data_booking['PAX']>0,:]
    
    #data_booking=data_booking.loc[data_booking['DEP_YEAR']==2019,:]
    
    # filter out duration langer than 29 nights
    data_booking=data_booking.loc[data_booking['BOOKED_DURATION']<30,:]
    
    data_booking['DEP_YEAR']=data_booking['CYCLE_START_DATE'].dt.year
    data_booking['BOOKING_YEAR']=data_booking['BOOKING_DATE'].dt.year
    data_booking['DIFF_YEAR']=data_booking['DEP_YEAR']-data_booking['BOOKING_YEAR']
    data_booking=data_booking.loc[data_booking['DIFF_YEAR']<=1,:]
    data_booking['YEAR_STR']='TY'
    data_booking['YEAR_STR'].loc[data_booking['DIFF_YEAR']==1]='LY'
    data_booking['BOOKING_YEAR_MONTH']=data_booking['YEAR_STR']+data_booking['BOOKING_DATE'].dt.strftime('%m')
    data_booking['DEP_WEEK']=data_booking['CYCLE_START_DATE'].dt.week
    data_booking=data_booking.loc[(data_booking['YEAR_STR']=='TY')|(data_booking['BOOKING_DATE'].dt.month>=4)]
    
        
    df0=data_booking.groupby(['BOOKING_YEAR_MONTH'],as_index=True)['PAX'].sum()
    df0_perc=df0.div(df0.sum())
    
    df1=pd.pivot_table(data_booking,values='PAX',index='BOOKING_YEAR_MONTH',columns='DEP_WEEK',aggfunc='sum')
    df1=df1.fillna(0)
    #df1.insert(0,df1.columns[0]-1,df1.iloc[:,0])
    df1_perc=df1.div(df1.sum(axis=1),axis=0)
    
    df2=pd.pivot_table(data_booking,values='PAX',index=['DEP_WEEK','BOOKING_YEAR_MONTH'],columns='BOOKED_DURATION',aggfunc='sum')
    df2=df2.fillna(0)
    df2sum=df2.sum(axis=1)
    df2_perc=df2.div(df2.sum(axis=1),axis=0)
    
    df3=pd.pivot_table(data_booking,values='PAX',index=['DEP_WEEK'],columns='BOOKED_DURATION',aggfunc='sum')
    df3=df3.fillna(0)
    df3_perc=df3.div(df3.sum(axis=1),axis=0)
    
    for i in range(len(df2_perc)):
        if df2sum.iloc[i]<10:
            df2_perc.iloc[i,:]=df3_perc.loc[df3_perc.index==df2_perc.index[i][0],:].values[0]
            
    df2_perc=df2_perc.div(df2_perc.sum(axis=1),axis=0)
    
    df_cax_remain=np.append(df_cax.to_numpy(copy=True),np.zeros([30,4]),axis=0)
    
    return df0_perc,df1_perc,df2_perc,df_cax_remain,data_booking

## Update Capacity

In [472]:
def update_cx(df0_perc,df1_perc,df2_perc,dw,dur,bm,dw_p,i,df_cax_remain,pvt_remain,result):
    dw[i]=np.random.choice(df1_perc.columns,p=dw_p.iloc[i,1:])
    dw_bm=(dw[i],bm[i])
    dur[i]=np.random.choice(df2_perc.columns,p=df2_perc.iloc[[x for x in df2_perc.index.tolist()].index(dw_bm),:])
    if dur[i]>0:
        df_cax_sub=df_cax.loc[df_cax['WEEK']==dw[i],:]
        if len(df_cax_sub)>0:
            outbound_range=np.arange(df_cax_sub.index[0],df_cax_sub.index[-1]+1)
            bl=(df_cax_remain[outbound_range,1]>0) & (df_cax_remain[outbound_range+dur[i],2]>0) 
            df_cax_sub=df_cax_sub.loc[bl,:]
            bl2=(pvt_remain.loc[pvt_remain.index.isin(df_cax_sub['DP_DATE']),pvt_remain.columns==dur[i]]>0).values.reshape(-1,)
            df_cax_sub=df_cax_sub.loc[bl2,:]
            if len(df_cax_sub)>0:
                dp_date_index=np.random.choice(df_cax_sub.index,p=np.repeat(1/len(df_cax_sub),len(df_cax_sub)))
                df_cax_remain[dp_date_index,1]-=1
                df_cax_remain[dp_date_index+dur[i],2]-=1
                pvt_remain.loc[pvt_remain.index.date==df_cax_remain[dp_date_index,0],pvt_remain.columns==dur[i]]-=1
                result.append([i,1,bm[i],dw[i],dur[i],'Accepted!'])
            else: result.append([i,1,bm[i],dw[i],dur[i],'Rejected: within departure week, duration is not available!'])
        else: result.append([i,1,bm[i],dw[i],dur[i],'Rejected: departure week is not available!'])
    else: result.append([i,1,bm[i],dw[i],dur[i],'Rejected: duration is 0!'])
    return df_cax_remain,pvt_remain,result

## Plot 

In [473]:
def comparision_plot(result_sim,result_true,col_sim,col_true):
    result_sim.sort_values(by=col_sim,inplace=True)
    sim_temp=result_sim.groupby(col_sim,as_index=True)['npax'].sum()
    sim_perc=sim_temp.div(sim_temp.sum())
    
    result_true=result_true.loc[result_true['DEP_YEAR']==2019,:]
    #result_true=result_true.loc[result_true['PAX']>0,:]
    #result_true=result_true.loc[result_true['BOOKED_DURATION']<30,:]
    temp=result_true.groupby(col_true,as_index=True)['PAX'].sum()
    perc=temp.div(temp.sum())
    
    fig = make_subplots(rows=1, cols=2, subplot_titles=('Simulation '+col_sim,'True '+col_true))
    fig.add_trace(go.Bar(x=sim_perc.index,y=sim_perc), 
          row=1, col=1)
    fig.add_trace(go.Bar(x=perc.index,y=perc), 
          row=1, col=2)
    fig.update_layout(height=600, width=1000, title_text="Comparision_Plots")
    fig.show()

# Simulation

In [474]:
def Simulation (df0_perc,df1_perc,df2_perc,pvt,n_sim):
    lf=[None]*n_sim
    result_final=pd.DataFrame()
    for sim_iter in range(0,n_sim):
        # book month random draw
        bm=np.random.choice(a=df0_perc.index,size=ttl_pax_per_round,p=df0_perc)
        
        dw=[None]*ttl_pax_per_round
        dur=[None]*ttl_pax_per_round
            
        dw_p=pd.merge(pd.DataFrame(bm,columns=['bm']),df1_perc,right_on=df1_perc.index,left_on='bm',how='left')

        df_cax_remain=np.append(df_cax.to_numpy(copy=True),np.zeros([30,4]),axis=0)
        pvt_remain=pvt.copy()
        
        result=[]
        
        #fig, ax = plt.subplots()
        #y_max=1200
        
        for i in range(0,round(len(bm))):
            
            #ani_bm=pd.DataFrame([bm[i]]*len(df_cax_remain0),columns=['book_month'])
            df_cax_remain,pvt_remain,result=update_cx(df0_perc,df1_perc,df2_perc,dw,dur,bm,dw_p,i,df_cax_remain,pvt_remain,result)
            #df_temp=pd.DataFrame(df_cax_remain[:-30,:],columns=['dep_date','cax_o','cax_i','week'])
            
            #if (i%100==0)|(i==round(len(bm))):
             #   ax.clear()
              #  ax.set_ylim((0,y_max))
               # ax.set_autoscaley_on(False)
                #ax.bar(df_temp['dep_date'], df_temp['cax_o'], color='blue')
                #print(sum(df_temp['cax_o']))
                #fig.canvas.draw()
            #plt.pause(0.001)
        #print(1-sum(df_cax_remain0[:,1]/sum(df_cax['CAX_O'])))
 
        lf[sim_iter]=1-sum(df_cax_remain[:,1]/sum(df_cax['CAX_O']))

        result=pd.DataFrame(result)
        result_final=pd.concat([result_final,result], ignore_index=True)
    return result_final,lf,df_cax_remain,pvt_remain

# Capacity UK S19

In [475]:
df_cax=data_cax[['TRANSPORT_ARRIVAL_DATETIME','TRANSPORT_ALLOCATION']]
df_cax['TRANSPORT_ARRIVAL_DATETIME']=df_cax['TRANSPORT_ARRIVAL_DATETIME'].dt.date
df_cax=df_cax.groupby(['TRANSPORT_ARRIVAL_DATETIME'],as_index=False)['TRANSPORT_ALLOCATION'].sum()
daterange=pd.DataFrame(pd.date_range(start=min(df_cax['TRANSPORT_ARRIVAL_DATETIME']), end=max(df_cax['TRANSPORT_ARRIVAL_DATETIME'])).to_pydatetime().tolist(),columns=['TRANSPORT_ARRIVAL_DATETIME'])
daterange['TRANSPORT_ARRIVAL_DATETIME']=daterange['TRANSPORT_ARRIVAL_DATETIME'].dt.date
df_cax=pd.merge(daterange,df_cax,on='TRANSPORT_ARRIVAL_DATETIME',how='left')
df_cax=df_cax[['TRANSPORT_ARRIVAL_DATETIME','TRANSPORT_ALLOCATION']]
df_cax=df_cax.fillna(0)
df_cax['CAX_I']=df_cax['TRANSPORT_ALLOCATION']
df_cax['WEEK']=pd.to_datetime(df_cax['TRANSPORT_ARRIVAL_DATETIME']).dt.week
df_cax.columns=['DP_DATE','CAX_O','CAX_I','WEEK']

ttl_pax_per_round=round(sum(df_cax['CAX_O'])*ratio)
print('TTL Pax Per Round: '+str(ttl_pax_per_round))

TTL Pax Per Round: 100528


# Simulation Based on UK Booking Data

In [476]:
df0_perc_UK,df1_perc_UK,df2_perc_UK,df_cax_remain_UK,data_booking_UK=preprocessing(data_booking_UK)

In [477]:
plt.rcParams['figure.figsize'] = (5,6)
t=time.time()
result_final_UK,lf_UK,df_cax_remain_UK,pvt_remain_UK=Simulation (df0_perc_UK,df1_perc_UK,df2_perc_UK,pvt,n_sim)
df_cax_remain_UK=pd.DataFrame(df_cax_remain_UK[:-30,:],columns=df_cax.columns)
print(time.time()-t)

528.2115564346313


In [478]:
fig = make_subplots(rows=2, cols=2,shared_yaxes=True,subplot_titles=('Outbound Original Cax',
                                                                     'Outbound Free Cax',
                                                                    'Inbound Original Cax',
                                                                    'Inbound Free Cax'))
fig.add_trace(
    go.Bar(x=df_cax['DP_DATE'], y=df_cax['CAX_O'],marker=dict(color="green")),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=df_cax['DP_DATE'], y=df_cax['CAX_I'],marker=dict(color="green")),
    row=2, col=1
)

fig.add_trace(
    go.Bar(x=df_cax['DP_DATE'], y=df_cax_remain_UK['CAX_O'],marker=dict(color="magenta")),
    row=1, col=2
)

fig.add_trace(
    go.Bar(x=df_cax['DP_DATE'], y=df_cax_remain_UK['CAX_I'],marker=dict(color="magenta")),
    row=2, col=2
)
fig.update_yaxes(autorange="reversed", row=2, col=1)
fig.update_yaxes(autorange="reversed", row=2, col=2)


fig.update_layout(height=600, width=1000, title_text="Capacity before and after simulation",showlegend=False)
fig.show(),

(None,)

In [481]:
result_final_UK.columns=['id','npax','book_month','departure_week','duration','message']
result_final_UK.to_csv("C:/Users/changjin.feng/Desktop/dc/result_UK_with_DC.csv",index=False)
accepted_UK=result_final_UK.loc[result_final_UK['message'].str.contains('Accepted!', regex=True),:]

In [482]:
comparision_plot(accepted_UK,data_booking_UK,'book_month','BOOKING_YEAR_MONTH')

In [483]:
comparision_plot(accepted_UK,data_booking_UK,'departure_week','DEP_WEEK')

In [484]:
comparision_plot(accepted_UK,data_booking_UK,'duration','BOOKED_DURATION')

In [485]:
lf_UK

[0.8211244628362246]

In [486]:
accepted_UK.groupby(['duration']).npax.sum()

duration
3       239
4       537
7     28230
10     3229
11     2693
14     6329
21       12
28        4
Name: npax, dtype: int64

# Function push

In [487]:
def push(cax,duration):
    cax_new=cax.copy()
    outbound=cax_new['CAX_O'].iloc[:-duration].values
    inbound=cax_new['CAX_I'].iloc[duration:].values
    mtv=np.minimum(outbound,inbound)
    cax_new['mtv_o']=np.append(mtv,np.zeros(duration),axis=0)
    cax_new['mtv_i']=np.append(np.zeros(duration),mtv,axis=0)
    cax_new['CAX_O']=cax_new['CAX_O']-cax_new['mtv_o']
    cax_new['CAX_I']=cax_new['CAX_I']-cax_new['mtv_i']
    return cax_new

In [488]:
cax=df_cax_remain_UK.copy()

In [489]:
sum(cax['CAX_O'])

8991.0

In [490]:
cax

,DP_DATE,CAX_O,CAX_I,WEEK
0,2019-05-01,0,221,18
1,2019-05-02,0,0,18
2,2019-05-03,0,0,18
3,2019-05-04,2,189,18
4,2019-05-05,10,334,18
5,2019-05-06,0,0,19
6,2019-05-07,2,186,19
7,2019-05-08,2,55,19
8,2019-05-09,0,0,19
9,2019-05-10,0,0,19


In [491]:
pvt

DURATION,0,3,4,7,8,10,11,14,21,28
CYCLE_START_DATE,,,,,,,,,,
2019-05-01,0.0,0.0,11.0,160.0,0.0,0.0,24.0,26.0,0.0,0.0
2019-05-04,2.0,0.0,0.0,165.0,0.0,0.0,0.0,22.0,0.0,0.0
2019-05-05,10.0,6.0,0.0,283.0,0.0,24.0,0.0,22.0,0.0,0.0
2019-05-07,0.0,0.0,0.0,116.0,0.0,0.0,33.0,37.0,0.0,0.0
2019-05-08,2.0,0.0,0.0,155.0,0.0,10.0,32.0,22.0,0.0,0.0
2019-05-11,6.0,6.0,6.0,347.0,0.0,33.0,28.0,77.0,2.0,0.0
2019-05-12,6.0,6.0,0.0,272.0,0.0,27.0,0.0,34.0,0.0,0.0
2019-05-14,10.0,0.0,6.0,116.0,0.0,0.0,32.0,22.0,0.0,0.0
2019-05-15,4.0,1.0,13.0,125.0,0.0,36.0,17.0,25.0,0.0,0.0


## push 7 days

In [492]:
cax_after_push7=push(df_cax_remain_UK,7)

In [493]:
sum(cax_after_push7['CAX_O'])

4413.0

In [494]:
sim7=sum(accepted_UK.loc[accepted_UK['duration']==7,'npax'])
push7=sum(cax['CAX_O'])-sum(cax_after_push7['CAX_O'])
print("Simu 7: " +str(sim7) + "\n" + "Push 7: "+str(int(push7))+ "\n" + "TTL  7: "+str(int(sim7+push7)))

Simu 7: 28230
Push 7: 4578
TTL  7: 32808


## push 14 days

In [495]:
cax_after_push14=push(cax_after_push7,14)

In [496]:
sum(cax_after_push14['CAX_O'])

4290.0

In [497]:
sim14=sum(accepted_UK.loc[accepted_UK['duration']==14,'npax'])
push14=sum(cax_after_push7['CAX_O'])-sum(cax_after_push14['CAX_O'])
print("Simu 14: " +str(sim14) + "\n" + "Push 14: "+str(int(push14))+ "\n" + "TTL  14: "+str(int(sim14+push14)))

Simu 14: 6329
Push 14: 123
TTL  14: 6452


## push 10 days

In [498]:
cax_after_push10=push(cax_after_push14,10)

In [499]:
sum(cax_after_push10['CAX_O'])

4273.0

In [500]:
sim10=sum(accepted_UK.loc[accepted_UK['duration']==10,'npax'])
push10=sum(cax_after_push14['CAX_O'])-sum(cax_after_push10['CAX_O'])
print("Simu 10: " +str(sim10) + "\n" + "Push 10: "+str(int(push10))+ "\n" + "TTL  10: "+str(int(sim10+push10)))

Simu 10: 3229
Push 10: 17
TTL  10: 3246


## push 11 days

In [501]:
cax_after_push11=push(cax_after_push10,11)

In [502]:
sum(cax_after_push11['CAX_O'])

4228.0

In [503]:
sim11=sum(accepted_UK.loc[accepted_UK['duration']==11,'npax'])
push11=sum(cax_after_push10['CAX_O'])-sum(cax_after_push11['CAX_O'])
print("Simu 11: " +str(sim11) + "\n" + "Push 11: "+str(int(push11))+ "\n" + "TTL  11: "+str(int(sim11+push11)))

Simu 11: 2693
Push 11: 45
TTL  11: 2738


## Final LF 

In [504]:
1-sum(cax_after_push11['CAX_O'])/sum(df_cax['CAX_O'])

0.9158841317841795

# Final Comparision

In [505]:
df_push=pd.DataFrame([[7,push7],[14, push14],[10,push10],[11,push11]],columns=['DURATION','PAX_PUSH'])
df_push

,DURATION,PAX_PUSH
0,7,4578.0
1,14,123.0
2,10,17.0
3,11,45.0


In [506]:
df_sim=pd.DataFrame(accepted_UK.groupby(['duration'],as_index=False).npax.sum())
df_sim.columns=['DURATION','PAX_SIM']
df_sim

,DURATION,PAX_SIM
0,3,239
1,4,537
2,7,28230
3,10,3229
4,11,2693
5,14,6329
6,21,12
7,28,4


In [507]:
df_ttl=pd.merge(df_sim,df_push,how='left',on='DURATION')
df_ttl=df_ttl.fillna(0)
df_ttl['PAX']=df_ttl['PAX_SIM']+df_ttl['PAX_PUSH']
df_ttl.drop(['PAX_SIM','PAX_PUSH'],axis=1,inplace=True)
df_ttl['%SIM']=df_ttl['PAX']/sum(df_ttl['PAX'])
df_ttl

,DURATION,PAX,%SIM
0,3,239.0,0.005192
1,4,537.0,0.011665
2,7,32808.0,0.712660
3,10,3246.0,0.070510
4,11,2738.0,0.059475
5,14,6452.0,0.140151
6,21,12.0,0.000261
7,28,4.0,0.000087


In [508]:
df_true=pd.DataFrame(data_dcax_UK.groupby(['DURATION'],as_index=False).BOOKED_ALLOCATION.sum())
df_true['%TRUE']=df_true['BOOKED_ALLOCATION']/sum(df_true['BOOKED_ALLOCATION'])
df_true

,DURATION,BOOKED_ALLOCATION,%TRUE
0,0,2434,0.050088
1,3,258,0.005309
2,4,591,0.012162
3,7,31298,0.644071
4,8,178,0.003663
5,10,3782,0.077829
6,11,2985,0.061427
7,14,7033,0.144730
8,21,30,0.000617
9,28,5,0.000103


In [519]:
df_compare=pd.merge(df_true,df_ttl,how='left',on='DURATION')
df_compare=df_compare.fillna(0)
df_compare.columns=['DURATION','TRUE_PAX',"%TRUE","SIM_PAX","%SIM"]

In [521]:
df_compare

,DURATION,TRUE_PAX,%TRUE,SIM_PAX,%SIM
0,0,2434,0.050088,0.0,0.000000
1,3,258,0.005309,239.0,0.005192
2,4,591,0.012162,537.0,0.011665
3,7,31298,0.644071,32808.0,0.712660
4,8,178,0.003663,0.0,0.000000
5,10,3782,0.077829,3246.0,0.070510
6,11,2985,0.061427,2738.0,0.059475
7,14,7033,0.144730,6452.0,0.140151
8,21,30,0.000617,12.0,0.000261
9,28,5,0.000103,4.0,0.000087
